# Tik Tok Case Study
Author: Brent Artuch <br>
Date: 2023-06-17

## Introduction

This case study is one of the projects from the Google Advanced Data Analytics Certifications.

<b> Goal: </b>
The TikTok data team is developing a machine learning model for classifying claims made in videos submitted to the platform.

<b> Scenario: </b>
TikTok is the leading destination for short-form mobile video. The platform is built to help imaginations thrive. TikTok's
mission is to create a place for inclusive, joyful, and authentic content–where people can safely discover, create, and connect.

<b> Background on the TikTok Scenario: </b>
At TikTok, our mission is to inspire creativity and bring joy. Our employees lead with curiosity and move at the speed of culture.
Combined with our company's flat structure, you'll be given dynamic opportunities to make a real impact on a rapidly expanding
company and grow your career.

TikTok users have the ability to submit reports that identify videos and comments that contain user claims. These reports identify
content that needs to be reviewed by moderators. The process generates a large number of user reports that are challenging to
consider in a timely manner.

TikTok is working on the development of a predictive model that can determine whether a video contains a claim or offers an
opinion. With a successful prediction model, TikTok can reduce the backlog of user reports and prioritize them more efficiently.

<b> Note: </b> <i> The story, all names, characters, and incidents portrayed in this project are fictitious. No identification with
actual persons (living or deceased) is intended or should be inferred. And, the data shared in this project has been created for
pedagogical purposes. </i>

## Step 1: Imports
### Import libraries

In [1]:
import pandas as pd
import numpy as np

### Load the dataset.

In [3]:
data = pd.read_csv("dataset/tiktok_dataset.csv")

## Step 2: Inspect Data


In [4]:
data.head(10)

,#,claim_status,video_id,video_duration_sec,video_transcription_text,verified_status,author_ban_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,1,claim,7017666017,59,someone shared with me that drone deliveries a...,not verified,under review,343296.0,19425.0,241.0,1.0,0.0
1,2,claim,4014381136,32,someone shared with me that there are more mic...,not verified,active,140877.0,77355.0,19034.0,1161.0,684.0
2,3,claim,9859838091,31,someone shared with me that american industria...,not verified,active,902185.0,97690.0,2858.0,833.0,329.0
3,4,claim,1866847991,25,someone shared with me that the metro of st. p...,not verified,active,437506.0,239954.0,34812.0,1234.0,584.0
4,5,claim,7105231098,19,someone shared with me that the number of busi...,not verified,active,56167.0,34987.0,4110.0,547.0,152.0
5,6,claim,8972200955,35,someone shared with me that gross domestic pro...,not verified,under review,336647.0,175546.0,62303.0,4293.0,1857.0
6,7,claim,4958886992,16,someone shared with me that elvis presley has ...,not verified,active,750345.0,486192.0,193911.0,8616.0,5446.0
7,8,claim,2270982263,41,someone shared with me that the best selling s...,not verified,active,547532.0,1072.0,50.0,22.0,11.0
8,9,claim,5235769692,50,someone shared with me that about half of the ...,not verified,active,24819.0,10160.0,1050.0,53.0,27.0
9,10,claim,4660861094,45,someone shared with me that it would take a 50...,verified,active,931587.0,171051.0,67739.0,4104.0,2540.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19382 entries, 0 to 19381
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   #                         19382 non-null  int64  
 1   claim_status              19084 non-null  object 
 2   video_id                  19382 non-null  int64  
 3   video_duration_sec        19382 non-null  int64  
 4   video_transcription_text  19084 non-null  object 
 5   verified_status           19382 non-null  object 
 6   author_ban_status         19382 non-null  object 
 7   video_view_count          19084 non-null  float64
 8   video_like_count          19084 non-null  float64
 9   video_share_count         19084 non-null  float64
 10  video_download_count      19084 non-null  float64
 11  video_comment_count       19084 non-null  float64
dtypes: float64(5), int64(3), object(4)
memory usage: 1.8+ MB


In [7]:
data[['video_duration_sec', 'video_view_count', 'video_like_count', 'video_share_count', 'video_download_count', 'video_comment_count']].describe()

,video_duration_sec,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
count,19382.000000,19084.000000,19084.000000,19084.000000,19084.000000,19084.000000
mean,32.421732,254708.558688,84304.636030,16735.248323,1049.429627,349.312146
std,16.229967,322893.280814,133420.546814,32036.174350,2004.299894,799.638865
min,5.000000,20.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,4942.500000,810.750000,115.000000,7.000000,1.000000
50%,32.000000,9954.500000,3403.500000,717.000000,46.000000,9.000000
75%,47.000000,504327.000000,125020.000000,18222.000000,1156.250000,292.000000
max,60.000000,999817.000000,657830.000000,256130.000000,14994.000000,9599.000000


The ultimate goal is to use a machine learning model to classify videos as claims or opinions, so we
should examine the claims variable.

In [8]:
data['claim_status'].value_counts()

claim      9608
opinion    9476
Name: claim_status, dtype: int64

Next, examine trends associated with the two claim statuses.

In [14]:
avg_view_count_claim = round(data[data['claim_status']=='claim']['video_view_count'].mean(), 2)
print("Average view count claim:", avg_view_count_claim)

median_view_count_claim = round(data[data['claim_status']=='claim']['video_view_count'].median())
print("Median view count claim:", median_view_count_claim)

Average view count claim: 501029.45
Median view count claim: 501555


In [15]:
avg_view_count_opinion = round(data[data['claim_status']=='opinion']['video_view_count'].mean(), 2)
print("Average view count claim:", avg_view_count_opinion)

median_view_count_opinion = round(data[data['claim_status']=='opinion']['video_view_count'].median())
print("Median view count claim:", median_view_count_opinion)

Average view count claim: 4956.43
Median view count claim: 4953


Now, examine trends associated with the ban status of the author.

In [21]:
data.groupby(['claim_status', 'author_ban_status']).count()[['#']]

#
claim_status author_ban_status      
claim        active             6566
             banned             1439
             under review       1603
opinion      active             8817
             banned              196
             under review        463

Calculate the median video share count of each author ban status.

In [22]:
data.groupby(['author_ban_status']).agg(
    {'video_view_count': ['mean', 'median'],
     'video_like_count': ['mean', 'median'],
     'video_share_count': ['mean', 'median']}
)

video_view_count           video_like_count            \
                              mean    median             mean    median   
author_ban_status                                                         
active               215927.039524    8616.0     71036.533836    2222.0   
banned               445845.439144  448201.0    153017.236697  105573.0   
under review         392204.836399  365245.5    128718.050339   71204.5   

                  video_share_count           
                               mean   median  
author_ban_status                             
active                 14111.466164    437.0  
banned                 29998.942508  14468.0  
under review           25774.696999   9444.0

In [23]:
data.groupby(['author_ban_status']).median(numeric_only=True)[['video_share_count']]

,video_share_count
author_ban_status,
active,437.0
banned,14468.0
under review,9444.0


Banned authors have a median share count that is 33 times the median share count of active authors.
Add the count to the table.

In [24]:
data.groupby(['author_ban_status']).agg(
    {'video_view_count': ['count', 'mean', 'median'],
     'video_like_count': ['count', 'mean', 'median'],
     'video_share_count': ['count', 'mean', 'median']}
)

video_view_count                          video_like_count  \
                             count           mean    median            count   
author_ban_status                                                              
active                       15383  215927.039524    8616.0            15383   
banned                        1635  445845.439144  448201.0             1635   
under review                  2066  392204.836399  365245.5             2066   

                                           video_share_count                \
                            mean    median             count          mean   
author_ban_status                                                            
active              71036.533836    2222.0             15383  14111.466164   
banned             153017.236697  105573.0              1635  29998.942508   
under review       128718.050339   71204.5              2066  25774.696999   

                            
                    median  
author_ban_status           
active               437.0  
banned             14468.0  
under review        9444.0

Now, create three new columns to help better understand engagement rates:
* `likes_per_view`: represents the number of likes divided by the number of views for each video
* `comments_per_view`: represents the number of comments divided by the number of views for each video
* `shares_per_view`: represents the number of shares divided by the number of views for each video

In [25]:
data['likes_per_view'] = data['video_like_count'] / data['video_view_count']
data['comments_per_view'] = data['video_comment_count'] / data['video_view_count']
data['shares_per_view'] = data['video_share_count'] / data['video_view_count']

Then compile all the information in each of the three newly created columns for each combination
of categories of claim status and author ban status.

In [26]:
data.groupby(['claim_status', 'author_ban_status']).agg(
    {'likes_per_view': ['count', 'mean', 'median'],
     'comments_per_view': ['count', 'mean', 'median'],
     'shares_per_view': ['count', 'mean', 'median']}
)

likes_per_view                      \
                                        count      mean    median   
claim_status author_ban_status                                      
claim        active                      6566  0.329542  0.326538   
             banned                      1439  0.345071  0.358909   
             under review                1603  0.327997  0.320867   
opinion      active                      8817  0.219744  0.218330   
             banned                       196  0.206868  0.198483   
             under review                 463  0.226394  0.228051   

                               comments_per_view                      \
                                           count      mean    median   
claim_status author_ban_status                                         
claim        active                         6566  0.001393  0.000776   
             banned                         1439  0.001377  0.000746   
             under review                   1603  0.001367  0.000789   
opinion      active                         8817  0.000517  0.000252   
             banned                          196  0.000434  0.000193   
             under review                    463  0.000536  0.000293   

                               shares_per_view                      
                                         count      mean    median  
claim_status author_ban_status                                      
claim        active                       6566  0.065456  0.049279  
             banned                       1439  0.067893  0.051606  
             under review                 1603  0.065733  0.049967  
opinion      active                       8817  0.043729  0.032405  
             banned                        196  0.040531  0.030728  
             under review                  463  0.044472  0.035027